In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 

PATH = "/Users/jordandass/Desktop/Drivers/chromedriver"

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://0e0c55ie39.execute-api.eu-central-1.amazonaws.com/default/fplAnalytics-DownloadPlayerStatusData")

time.sleep(10)

df = pd.read_csv (r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')

send2trash(r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')

name_list = []
cost_list = []
total_points_list = []
position_list = []
ppg_list = []
team_list = []
points_cost_list = []
clean_name_list = []

time_now = re.findall('[0-9]{4}-[0-9]+-[0-9]+', str(datetime.now()))[0]

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

for num in range(len(df)):
    if int(df.iloc[num, 6]) != 0:
        if str(df.iloc[num, 4]) != 'NotAvail':
            name_list.append(df.iloc[num, 0])
            cost_list.append(df.iloc[num, 3])
            total_points_list.append(df.iloc[num, 6])
            position_list.append(df.iloc[num, 2])
            ppg_list.append(df.iloc[num, 8])
            team_list.append(df.iloc[num, 1])
            points_cost_list.append(format(float(df.iloc[num, 6])/float(df.iloc[num, 3]),'.2f'))
            clean_name_list.append(strip_accents(str(df.iloc[num, 0]).lower()))
            
            
prices_df = pd.read_csv('prices.csv')
cost_list_ = []
for player in list(prices_df['Players']):
    cost_list_.append(list(df[df["name"] == player]["cost"])[0])
if list(prices_df.iloc[:,-1]) != cost_list_ or time_now in list(prices_df.columns):
    prices_df[time_now] = cost_list_
    prices_df.to_csv('prices.csv', index = False)


fpl_players = pd.DataFrame({'Players':name_list, 'Costs':cost_list, 'Total points':total_points_list, 'Positions':position_list, 'PPG':ppg_list, 'Team':team_list, 'Points per mil':points_cost_list, 'Clean name':clean_name_list})

fpl_players.to_csv('fpl_players.csv')
cost_effective = fpl_players.sort_values(by = 'Points per mil', ascending = False)
cost_effective.to_csv('cost_effective.csv')

gk_df = []
def_df = []
mid_df = []
fwd_df = []

for position in ['GKP','DEF','MID','FWD']:
    for num in range(len(fpl_players)):
        if fpl_players.iloc[num, 3] == position:
            if position == 'GKP':
                gk_df.append(list(fpl_players.iloc[num,]))
            elif position == 'DEF':
                def_df.append(list(fpl_players.iloc[num,]))
            elif position == 'MID':
                mid_df.append(list(fpl_players.iloc[num,]))
            elif position == 'FWD':
                fwd_df.append(list(fpl_players.iloc[num,]))
                

gk_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = gk_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
def_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = def_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
mid_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = mid_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
fwd_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = fwd_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])

best_df = pd.concat([gk_df, def_df, mid_df, fwd_df]).sort_values(by = 'Points per mil', ascending = False)
best_df.to_csv("best.csv")


gk_prices = sorted(list(set(list(gk_df.iloc[:,1]))))

def_prices = sorted(list(set(list(def_df.iloc[:,1]))))
    
mid_prices = sorted(list(set(list(mid_df.iloc[:,1]))))

fwd_prices = sorted(list(set(list(fwd_df.iloc[:,1]))))

all_price_quantity = {}
gk_price_quantity = {}
def_price_quantity = {}
mid_price_quantity = {}
fwd_price_quantity = {}


for price_list in [gk_prices, def_prices, mid_prices, fwd_prices]:
    for price in price_list:
        if price_list == gk_prices:
            gk_price_quantity[price] = len(gk_df[gk_df['Costs'] == price])
        elif price_list == def_prices:
            def_price_quantity[price] = len(def_df[def_df['Costs'] == price])
        elif price_list == mid_prices:
            mid_price_quantity[price] = len(mid_df[mid_df['Costs'] == price])
        elif price_list == fwd_prices:
            fwd_price_quantity[price] = len(fwd_df[fwd_df['Costs'] == price])    
            
    all_price_quantity['gk'] = gk_price_quantity
    all_price_quantity['def'] = def_price_quantity
    all_price_quantity['mid'] = mid_price_quantity
    all_price_quantity['fwd'] = fwd_price_quantity
driver.quit()



[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
[WDM] - Driver [/Users/jordandass/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [2]:
# Price Changes

import pandas as pd
from collections import defaultdict

def price_change():
    df = pd.read_csv("prices.csv")
    df2 = pd.read_csv("FPL_API.csv")
    df3 = pd.read_csv("best.csv")

    price_changes = defaultdict(list)
    for num in range(len(df)):
        if df.iloc[num, -1] != df.iloc[num, -2]:
            price_changes["Player"] += [df.iloc[num, 1]]
            price_changes["Price before"] += [df.iloc[num, -2]]
            price_changes["Price after"] += [df.iloc[num, -1]]
            if price_changes["Price before"][-1] > price_changes["Price after"][-1]:
                price_changes["Change"] += ["Fall"]
            else:
                price_changes["Change"] += ["Rise"]
            
            team = df3[df3["Players"] == df.iloc[num, 1]]
            
            price_changes["Team"] += [list(team["Team"])[0]]
            price_changes["Total Points"] += [list(team["Total points"])[0]]
            price_changes["Points per mil"] += [list(team["Points per mil"])[0]]
            

            player = df2[df2["Players"] == df.iloc[num, 1]]
            
            if len(player) != 1:
                convert = {"ARS":"Arsenal", "AVL":"Aston Villa", "BHA":"Brighton", "BUR":"Burnley", "CHE":"Chelsea", "CRY":"Crystal Palace", "EVE":"Everton",
                          "FUL":"Fulham", "LEI":"Leicester", "LIV":"Liverpool", "LEE":"Leeds", "MCI":"Man City", "MUN":"Man Utd", "NEW":"Newcastle",
                          "SOU":"Southampton", "BRE":"Brentford", "NOR":"Norwich", "WAT":"Watford", "TOT":"Spurs", "WHU":"West Ham", "WOL":"Wolves"}
                convert_team = convert[df.iloc[num, 3]]
                team_player = player[player["Team"] == convert_team]
                
                price_changes["Influence"] += [float(team_player["Influence"])]
                price_changes["Creativity"] += [float(team_player["Creativity"])]
                price_changes["Threat"] += [float(team_player["Threat"])]
                price_changes["ICT index"] += [float(team_player["ICT index"])]
                
            else:
                price_changes["Influence"] += [float(player["Influence"])]
                price_changes["Creativity"] += [float(player["Creativity"])]
                price_changes["Threat"] += [float(player["Threat"])]
                price_changes["ICT index"] += [float(player["ICT index"])]
    df = pd.DataFrame(price_changes).set_index("Change")
    return df

price_change()
        


TypeError: cannot convert the series to <class 'float'>

In [10]:
import unicodedata
from collections import defaultdict
import numpy as np
from itertools import product

def team_build(budget, my_team):
    x = None

    my_team = my_team
    my_team_list = []
    for t in range(len(my_team)):
        for player in my_team[t]:
            if player != None:
                my_team_list.append(player)


    goalkeepers = my_team[0]
    defenders = my_team[1]
    midfielders = my_team[2]
    fowards = my_team[3]

    fill_gkp = 0
    fill_def = 0
    fill_mid = 0
    fill_fwd = 0
    budget = float(budget)
    teams = defaultdict(int)

    for position in [goalkeepers, defenders, midfielders, fowards]:
        for player in position:
            if position == goalkeepers:
                if player == x:
                    fill_gkp += 1
                else:
                    budget -= float(gk_df[gk_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(gk_df[gk_df['Clean name'] == player.lower()]['Team'])[0]] += 1

            if position == defenders:
                if player == x:
                    fill_def += 1
                else:
                    budget -= float(def_df[def_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(def_df[def_df['Clean name'] == player.lower()]['Team'])[0]] += 1

            if position == midfielders:
                if player == x:
                    fill_mid += 1
                else:
                    budget -= float(mid_df[mid_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(mid_df[mid_df['Clean name'] == player.lower()]['Team'])[0]] += 1

            if position == fowards:
                if player == x:
                    fill_fwd += 1
                else:
                    budget -= float(fwd_df[fwd_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(fwd_df[fwd_df['Clean name'] == player.lower()]['Team'])[0]] += 1

    budget = format(float(budget), '.1f')
    print(budget)
    print(list(teams))
    print(my_team_list)

    full_teams = []
    for team in teams.keys():
        if teams[team] >= 3:
            full_teams.append(team)

    potential_gk = gk_df[~gk_df['Clean name'].isin(my_team_list) & ~gk_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)
    potential_def = def_df[~def_df['Clean name'].isin(my_team_list) & ~def_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)
    potential_mid = mid_df[~mid_df['Clean name'].isin(my_team_list) & ~mid_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)
    potential_fwd = fwd_df[~fwd_df['Clean name'].isin(my_team_list) & ~fwd_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)

    for fill in [fill_gkp, fill_def, fill_mid, fill_fwd]:
        min_gkp_cost = 0
        min_def_cost = 0
        min_mid_cost = 0
        min_fwd_cost = 0
        for num in range(fill):
            if fill == fill_gkp:
                min_gkp_cost += float(gk_df.iloc[num, 1])
            if fill == fill_def:
                min_def_cost += float(def_df.iloc[num, 1])
            if fill == fill_mid:
                min_mid_cost += float(mid_df.iloc[num, 1])
            if fill == fill_fwd:
                min_fwd_cost += float(fwd_df.iloc[num, 1])

    gk_comb = list(combinations(list(range(len(potential_gk))), fill_gkp))
    def_comb = list(combinations(list(range(len(potential_def))), fill_def))
    mid_comb = list(combinations(list(range(len(potential_mid))), fill_mid))
    fwd_comb = list(combinations(list(range(len(potential_fwd))), fill_fwd))

    gk_comb_poss = []
    def_comb_poss = []
    mid_comb_poss = []
    fwd_comb_poss = []
    for pos_comb in [gk_comb, def_comb, mid_comb, fwd_comb]:
        for comb in pos_comb:
            price = 0
            if pos_comb == gk_comb:
                for num in comb:
                    price += float(potential_gk.iloc[num, 1])
                if (float(budget) - price - min_def_cost - min_mid_cost - min_fwd_cost) >= 0:
                    gk_comb_poss.append(comb)
            elif pos_comb == def_comb:
                for num in comb:
                    price += float(potential_def.iloc[num, 1])
                if (float(budget) - price - min_gkp_cost - min_mid_cost - min_fwd_cost) >= 0:
                    def_comb_poss.append(comb)
            elif pos_comb == mid_comb:
                for num in comb:
                    price += float(potential_mid.iloc[num, 1])
                if (float(budget) - price - min_def_cost - min_gkp_cost - min_fwd_cost) >= 0:
                    mid_comb_poss.append(comb)
            elif pos_comb == gk_comb:
                for num in comb:
                    price += float(potential_fwd.iloc[num, 1])
                if (float(budget) - price - min_def_cost - min_mid_cost - min_gkp_cost) >= 0:
                    fwd_comb_poss.append(comb)


    def cartesian_product(arr1, arr2, arr3, arr4):
        for arr in [arr1, arr2, arr3, arr4]:
            if arr != []:
                arr = None
        return list(product(arr1, arr2, arr3, arr4))

    if gk_comb_poss == [] or gk_comb_poss == [(), ()]:
        gk_comb_poss = ['No gk']
    if def_comb_poss == [] or def_comb_poss == [(), ()]:
        def_comb_poss = ['No def']
    if mid_comb_poss == [] or mid_comb_poss == [(), ()]:
        mid_comb_poss = ['No mid']
    if fwd_comb_poss == [] or fwd_comb_poss == [(), ()]:
        fwd_comb_poss = ['No fwd']

    score_player = defaultdict(list)
    for pos_comb in cartesian_product(gk_comb_poss, def_comb_poss, mid_comb_poss, fwd_comb_poss):
        player_comb_num = 0
        highest_score = 0
        players = []
        for player_comb in pos_comb:
            for player_num in player_comb:
                if type(player_num) == int:
                    if player_comb == pos_comb[0]:
                        highest_score += float(potential_gk.iloc[player_num , 2])
                        players += [potential_gk.iloc[player_num, 0]]

                    elif player_comb == pos_comb[1]:
                        highest_score += float(potential_def.iloc[player_num, 2])
                        players += [potential_def.iloc[player_num, 0]]

                    elif player_comb == pos_comb[2]:
                        highest_score += float(potential_mid.iloc[player_num, 2])
                        players += [potential_mid.iloc[player_num, 0]]

                    elif player_comb == pos_comb[3]:
                        highest_score += float(potential_fwd.iloc[player_num, 2])
                        players += [potential_fwd.iloc[player_num, 0]]
                    player_comb_num += 1
        score_player[highest_score] += [players]

    final_comb = defaultdict(list)
    for score in score_player.keys():
        for player_comb in score_player[score]:
            spent = 0
            for player in player_comb:
                spent += list(best_df[best_df['Players'] == player]['Costs'])[0]
            if float(spent) <= float(budget):
                final_comb[score] += ([player_comb] + ['{:.1f}'.format(float(spent))])
    for key in sorted(list(final_comb.keys()), reverse = True):
        print(int(key))
        print(final_comb[key])
        print('')
        
x = None
team_build(101.6, 
           [['sanchez', 'steer'],
          ['rudiger', 'white', 'alexander-arnold', 'livramento', 'duffy'],
          ['salah', 'gray', x, x, 'douglas luiz'],
          ['ronaldo', 'lukaku','toney']])

12.5
['BHA', 'AVL', 'CHE', 'ARS', 'LIV', 'SOU', 'EVE', 'MUN', 'BRE']
['sanchez', 'steer', 'rudiger', 'white', 'alexander-arnold', 'livramento', 'duffy', 'salah', 'gray', 'douglas luiz', 'ronaldo', 'lukaku', 'toney']
106


TypeError: unhashable type: 'list'